## Tuning Blueberry Masks
#### Blueberry masks used by the classifier use the HSV colorspace to create numerical representations of the desired colors.
![HSV colorspace reference from Adobe](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fas1.ftcdn.net%2Fv2%2Fjpg%2F04%2F75%2F66%2F04%2F1000_F_475660473_MszZtgh9iFyLnDzJORs1M3WsrqMe4VUI.jpg&f=1&nofb=1&ipt=1a8f8bfe772c52ff9505d62ab02613859e0fab18799e99a3f42d3356db1b7758&ipo=images)
---
This Jupyter Notebook is written to help you find or tweak your HSV masks to better suit your berries.

1. import dependencies

In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import colors
import matplotlib.pyplot as plt

2. read your test image from your local files
    1. this can be a recent frame from the system's captured stream: 
    2. navigate to `/home/blueberryjam/BlueberryJam/logs/` in your file explorer and select an appropriate image with berries that should be classified

In [ ]:
# replace img_path with the image's file name
img_path = 'image_01.jpg'
img = cv.imread('/home/blueberryjam/BlueberryJam/logs/'+img_path)

# convert from BGR to other color spaces
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB) # this makes the image viewable to you
img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV) # this converts the image to the colorspace that the masks use

# display image in color spaces
fig = plt.figure()
gridspec = fig.add_gridspec(nrows=4, ncols=12)
fig.add_subplot(gridspec[0:4, 0:5])
plt.title("RGB Colorspace")
plt.imshow(img_rgb)
fig.add_subplot(gridspec[0:4, 7:12])
plt.title("HSV Colorspace")
plt.imshow(img_hsv)

3. Normalize the image's colors and split the image into separate Hue, Saturation, and Value channels

In [ ]:
# normalize colors
norm = colors.Normalize(vmin=-1.,vmax=1.)
pixel_colors = img.reshape((np.shape(img)[0]*np.shape(img)[1], 3))
norm.autoscale(pixel_colors)
pixel_colors = norm(pixel_colors).tolist()

hue,sat,val = cv.split(img)
hue = cv.normalize(hue,None,0,255,cv.NORM_MINMAX)

4. Show the image in each color channel.
    1. Set show_scatter according to your visualization preferences -- each setting has tradeoffs
        1. `show_scatter = True` 
        - 2D scatter plots show two color channels plotted on a 2D plane 
        - Advantage: compare pixel color against the numerical values for two channels at a time
        - Disadvantage: computationally expensive and slow to generate scatter plots
        2. `show_scatter = False`
        - Single-channel images show one HSV color channel values across an image in a heatmap style
        - Advantage: quick to generate single-channel images
        - Disadvantage: compare pixel color against colormap -- requires interpolation

In [ ]:
show_scatter = False

if show_scatter == True:
    plt.figure()
    plt.title('Underripe')
    plt.scatter(hue.flatten(), sat.flatten(), s=val.flatten(), c=pixel_colors, marker=".")
    plt.xlabel("Hue")
    plt.ylabel("Saturation")

    plt.figure()
    plt.scatter(sat.flatten(), val.flatten(), s=hue.flatten(), c=pixel_colors, marker=".")
    plt.xlabel("Saturation")
    plt.ylabel("Value")

    plt.figure()
    plt.scatter(val.flatten(), hue.flatten(), s=sat.flatten(), c=pixel_colors, marker=".")
    plt.xlabel("Value")
    plt.ylabel("Hue")
else:
    plt.figure(figsize=(30,30))
    plt.subplot(1,3,1)
    plt.title('Hue')
    plt.imshow(hue)
    plt.axis('off')
    plt.colorbar()
    plt.subplot(1,3,2)
    plt.title('Saturation')
    plt.imshow(sat)
    plt.axis('off')
    plt.colorbar()
    plt.subplot(1,3,3)
    plt.title('Value')
    plt.axis('off')
    plt.imshow(val)
    plt.colorbar()

![HSV reference image](./media/HSV_ref_img.png)

**Reference the images or plots above to inform your values below.**
| Channel | Effect |
| ----------- | ----------- |
| Hue | 0 is red; as values increase, the color follows the inverse rainbow until orange circles back to red again at 255| 
| Saturation | 0 is a shade of grey; as values increase, the color becomes more vibrant; 255 is the most vibrant | 
| Value | 0 is black; as values increase, the color becomes lighter | 

5. Test your values.
    1. Once you've explored the colors in your image, identify the parts that you would like to keep.
    2. Using whichever visualization you choose, note the high and low end for your desired areas in the Hue Channel. Type them below.

    

In [ ]:
# replace 0 and 255 with your high and low ends
hue_high = 255
hue_low = 0

3. Note the high and low end for the your desired areas in the Saturation Channel. Type them below.

In [ ]:
# replace 0 and 255 with your high and low ends
sat_high = 255
sat_low = 0

4. Note the high and low end for the your desired areas in the Value Channel. Type them below.

In [ ]:
# replace 0 and 255 with your high and low ends
val_high = 255
val_low = 0

5. Preview your mask's performance. Change the above values and re-run until that mask works adequately.

In [ ]:
mask_lows = (hue_low, sat_low, val_low)
mask_highs = (hue_high, sat_high, val_high)

mask = cv.inRange(img, mask_lows, mask_highs)
masked_rgb = cv.bitwise_and(img_rgb, img_rgb, mask=mask)

plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.xlabel('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(masked_rgb)
plt.xlabel('Masking Results')
plt.show()